# ATENTION!

Since TennisTV does not have the rights to show highlights of Grand Slams, we will need to take those videos from the official channels of the Grand Slams
- Australia Open TV
- US Open
- Wimbledon
- Rolang Garros

In [3]:
import os
os.chdir("/usr/src/app")
import pandas as pd

In [21]:
PATH_US_OPEN_VIDEOS = "deepsetstats/dataset/grandslams/parquet/us_open_videos.parquet"
PATH_AUSTRALIA_OPEN_VIDEOS = "deepsetstats/dataset/grandslams/parquet/australia_open_videos.parquet"
PATH_ROLANDGARROS_VIDEOS = "deepsetstats/dataset/grandslams/parquet/roland_garros_videos.parquet"
PATH_WIMBLEDON_VIDEOS = "deepsetstats/dataset/grandslams/parquet/wimbledon_videos.parquet"

In [22]:
df_us = pd.read_parquet(PATH_US_OPEN_VIDEOS, engine="pyarrow")
df_rg = pd.read_parquet(PATH_ROLANDGARROS_VIDEOS, engine="pyarrow")
df_ao = pd.read_parquet(PATH_AUSTRALIA_OPEN_VIDEOS, engine="pyarrow")
df_wb = pd.read_parquet(PATH_WIMBLEDON_VIDEOS, engine="pyarrow")

### Check the reference court videos


In [7]:
df_wb[df_wb["video_id"] == "dvBr9Wr8BCY"]

,title,video_id,date_publish
44,Carlos Alcaraz vs Novak Djokovic: Extended Hig...,dvBr9Wr8BCY,2023-07-17T15:18:50Z


In [8]:
df_ao[df_ao["video_id"] == "F9ZGyxlCyBU"]


,title,video_id,date_publish
1906,Rafael Nadal v Daniil Medvedev Condensed Match...,F9ZGyxlCyBU,2022-03-03T13:00:08Z


In [23]:
df_rg[df_rg["video_id"] == "QNgE9-0sNjQ&t"]

,title,video_id,date_publish
2600,Rafael Nadal v Stan Wawrinka Highlights - Men'...,QNgE9-0sNjQ&t,2017-06-11T20:55:31Z


In [24]:
df_us[df_us["video_id"] == "XmW-ArFVZJQ"]

,title,video_id,date_publish
350,Carlos Alcaraz vs. Frances Tiafoe Extended Hig...,XmW-ArFVZJQ,2022-09-11T05:00:11Z


# Get master table

In [25]:
import os

import pandas as pd
import tqdm

from deepsetstats.dataset.utils import (
    extract_players_title,
    extract_tournament_title,
    load_pickle,
)

os.system("clear")

# Path dataframe
PATH_US_OPEN_VIDEOS = "deepsetstats/dataset/grandslams/parquet/us_open_videos.parquet"
PATH_AUSTRALIA_OPEN_VIDEOS = "deepsetstats/dataset/grandslams/parquet/australia_open_videos.parquet"
PATH_ROLANDGARROS_VIDEOS = "deepsetstats/dataset/grandslams/parquet/roland_garros_videos.parquet"
PATH_WIMBLEDON_VIDEOS = "deepsetstats/dataset/grandslams/parquet/wimbledon_videos.parquet"
PATH_TOURNAMENTS_NAMING = (
    "deepsetstats/dataset/tournaments/parquet/tournaments_tennistv.parquet"
)

PATH_BIBLE_PLAYERS = "deepsetstats/dataset/players/parquet/bible_players.parquet"
PATH_MAP_ID2NAME = "deepsetstats/dataset/players/pickle/map_id2name.parquet"

# Output Path
PATH_MASTER = "deepsetstats/dataset/grandslams/parquet/master.parquet"

In [49]:
# --------------------------------------- #
# --------------------------------------- #
# Get datasets
# --------------------------------------- #
# --------------------------------------- #
df_tour = pd.read_parquet(PATH_TOURNAMENTS_NAMING, engine="pyarrow")
df_us = pd.read_parquet(PATH_US_OPEN_VIDEOS, engine="pyarrow")
df_rg = pd.read_parquet(PATH_ROLANDGARROS_VIDEOS, engine="pyarrow")
df_ao = pd.read_parquet(PATH_AUSTRALIA_OPEN_VIDEOS, engine="pyarrow")
df_wb = pd.read_parquet(PATH_WIMBLEDON_VIDEOS, engine="pyarrow")
df_players = pd.read_parquet(PATH_BIBLE_PLAYERS, engine="pyarrow")
df_players = df_players[df_players["best_ranking"] < 100].copy()
d1 = load_pickle(PATH_MAP_ID2NAME)


df_us["tournament_name"] = "US Open"
df_rg["tournament_name"] = "Roland Garros"
df_ao["tournament_name"] = "Australian Open"
df_wb["tournament_name"] = "Wimbledon"

# --------------------------------- #
# Players
# --------------------------------- #
player_common_names = df_players["common_name"].tolist()
player_common_names_2 = df_players["common_name_2"].tolist()
player_full_names = df_players["name"].tolist()
player_ids = df_players["player_id"].tolist()

# --------------------------------- #
# Ids of Grand Slams
# --------------------------------- #
d_ids_grandslams = {} 
for i, row in df_tour[df_tour["level"] == "grandslam"].iterrows():
    d_ids_grandslams[row["tournament_name"]] = row["tournament_id"]

# --------------------------------- #
# Tennis TV Get Videos of Highlights
# --------------------------------- #
df_all_grands = pd.concat([df_us, df_rg, df_ao, df_wb])
mask1 = df_all_grands.title.str.contains("ighlights")
mask2 = df_all_grands.title.str.contains("Condensed Match")
mask3 = df_all_grands.title.str.contains("Full Match")
dfh = df_all_grands[mask1 | mask2 | mask3]

# --------------------------------------- #
# --------------------------------------- #
# Create master table
# --------------------------------------- #
# --------------------------------------- #
ept = extract_players_title
ett = extract_tournament_title

d_master = {
    "video_id": [],
    "player_id": [],
    "tournament_id": [],
}
not_found_tourn = []
not_found_players = []

In [50]:
for _, row in tqdm.tqdm(dfh.iterrows()):
    video_id = row["video_id"]
    raw_title = row["title"]
    tour_name = row["tournament_name"]
    tour_id = d_ids_grandslams[tour_name]

    play_ids = ept(
        raw_string=raw_title,
        player_ids=player_ids,
        player_full_names=player_full_names,
        player_common_names=player_common_names,
        player_common_names_2=player_common_names_2,
    )

    if len(play_ids) < 1:
        not_found_players.append(video_id)
        continue

    for pid in play_ids:
        d_master["video_id"].append(video_id)
        d_master["player_id"].append(pid)
        d_master["tournament_id"].append(tour_id)

3505it [10:47,  5.41it/s]


In [51]:
df_final = pd.DataFrame(d_master)

cols_tour = ["tournament_name", "tournament_id"]
df_final = pd.merge(df_final, df_tour[cols_tour], on="tournament_id", how="left")

cols_players = ["player_id", "name"]
df_final = pd.merge(df_final, df_players[cols_players], on="player_id", how="left")

cols_tt = ["title", "video_id"]
df_final = pd.merge(df_final, dfh[cols_tt], on="video_id", how="left")

# Writing final dataframe master
df_final.to_parquet(PATH_MASTER, engine="pyarrow")

In [53]:
df_final

,video_id,player_id,tournament_id,tournament_name,name,title
0,k5Azrtqr_ug,123,3,US Open,Fabio Fognini,Jakub Mensik vs. Fabio Fognini Highlights | 20...
1,WdaEsoL4_ak,166,3,US Open,Vasek Pospisil,Vasek Pospisil vs. Pedro Martinez Highlights |...
2,WdaEsoL4_ak,4145,3,US Open,Alberto Martin,Vasek Pospisil vs. Pedro Martinez Highlights |...
3,TGR4PnD0cnM,162,3,US Open,Denis Kudla,Tennys Sandgren vs. Denis Kudla Highlights | 2...
4,TGR4PnD0cnM,217,3,US Open,Tennys Sandgren,Tennys Sandgren vs. Denis Kudla Highlights | 2...
...,...,...,...,...,...,...
3456,KgTLdOI-CfA,3140,2,Wimbledon,Benjamin Becker,2013 Day 1 Highlights: Andy Murray v Benjamin ...
3457,6f1YyGnldJk,138,2,Wimbledon,Rafael Nadal,2013 Day 1 Highlights: Rafael Nadal v Steve Da...
3458,6f1YyGnldJk,2386,2,Wimbledon,Steve Darcis,2013 Day 1 Highlights: Rafael Nadal v Steve Da...
3459,dCs_NNf2RmM,2059,2,Wimbledon,Roger Federer,2013 Day 1 Highlights: Roger Federer v Victor ...


In [54]:
df_final[df_final["video_id"] == "dvBr9Wr8BCY"]

,video_id,player_id,tournament_id,tournament_name,name,title
2693,dvBr9Wr8BCY,0,2,Wimbledon,Carlos Alcaraz,Carlos Alcaraz vs Novak Djokovic: Extended Hig...
2694,dvBr9Wr8BCY,1,2,Wimbledon,Novak Djokovic,Carlos Alcaraz vs Novak Djokovic: Extended Hig...
